In [162]:
# Library for opening url and creating
# requests
import urllib.request
import requests
 
# pretty-print python data structures
from pprint import pprint
 
# for parsing all the tables present
# on the website
from html_table_parser.parser import HTMLTableParser
 
# for converting the parsed data in a
# pandas dataframe
import pandas as pd

from typing import List

import csv
import pathlib

import bs4
from bs4 import BeautifulSoup

# Need to use multiprocess instead of multiprocessing for IPython implementations
# https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
import multiprocess as mp

In [130]:
k_output_directory = pathlib.Path.cwd()
overwrite_cache = False

In [251]:
def get_ship_urls(ships_url: str, max_page_number: int = -1):
    """
    Finds all the ship urls from the ship main page. 
    https://www.cruisemapper.com/ships?page=0
    
    Will scrape all the urls for all ships on this page and then go to the next page. Keeps going until
    it hits the last page (at which point all ships have been scraped) and then stops. Alternatively, 
    stops if it hits the last page number requested.
    """
    page_number = ships_url.split("=")[-1]
    
    response = url_get_contents(ships_url)
    soup = BeautifulSoup(response,"html.parser")
    
    # Get all the ship list items. 
    ship_lists = soup.find_all("div", {"class": "shipListItemContent"})
    
    print("Found {} ships on page {}...".format(len(ship_lists), page_number))
    
    output_urls = [ship_item.find("h3").a["href"] for ship_item in ship_lists]
        
    next_url = soup.find("ul", {"class": "pager"}).find_all("li")[-1].a["href"]
    
    if next_url == ships_url:
        print("hit last page... no more pages to scrape")
        return output_urls
    elif max_page_number >= 0 and page_number >= max_page_number:
        print("hit max page number requested... stopping")
        return output_urls
    
    return output_urls + get_ship_urls(next_url)

def ship_url_cache_file_name():     
    return "{}/ship_cache/ship_cache.csv".format(k_output_directory)
    
def cache_ship_urls(ship_url_list: List[str], cache_file: str):    
    print("Attempting to cache all ship urls to {}".format(cache_file))
    output_dir = pathlib.Path(cache_file).parents[0]
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    with open(cache_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for ship_url in ship_url_list:
            csv_writer.writerow([ship_url])

def load_cache_ship_urls(cache_file: str):
    
    if not pathlib.Path(cache_file).is_file():
        return None
    
    with open(cache_file, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        output = [ship_url[0] for ship_url in csv_reader]
        
        print("Load from cache successful!")
        return output

def load_all_ship_urls():
    all_ship_urls = load_cache_ship_urls(ship_url_cache_file_name())

    if not all_ship_urls or overwrite_cache:
        all_ship_urls = get_ship_urls("https://www.cruisemapper.com/ships?page=0")
        cache_ship_urls(all_ship_urls, ship_url_cache_file_name())

    print("Loaded {} total ship urls!".format(len(all_ship_urls)))
    return all_ship_urls

In [244]:
def get_json_itinerary_contents_from_id(itinerary_id: int, referer_url: str):
    '''
    Opens a website and read its binary contents (HTTP Response Body)
    '''
    query_url = "https://www.cruisemapper.com/ships/cruise.json?id={}".format(itinerary_id)
    
    # This header is required, otherwise the website will reject the request with a 403 error.
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15'
    headers={
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9', 
        #'Referer':referer_url,
        'Host': 'www.cruisemapper.com', 
        'User-Agent':user_agent,
        'Cookie': '_ga=GA1.1.768899093.1682051958; _ga_S7E9PPFQCS=GS1.1.1682086656.3.1.1682087164.0.0.0; __gpi=UID=00000be4190695e2:T=1682051957:RT=1682086656:S=ALNI_MaK8BStwoBIl9_a-fR4Lwgd0oA-rQ; __gads=ID=3f180da5ffcf9cc8-22e37cdd50df0055:T=1682051957:RT=1682051957:S=ALNI_MavQUFDp5P3x373PKqQFDTmxBkyFQ',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'X-Requested-With': 'XMLHttpRequest'
    } 
 
    # Opens a website and read its
    # binary contents (HTTP Response Body)
 
    # making request to the website
    #req = urllib.request.Request(url, None, headers)
    # f = urllib.request.urlopen(req)
    f = requests.get(query_url, headers=headers)
    #print(dir(f))
    #print(f.json())
    #pprint(f.raw)
 
    #reading contents of the website
    return f

def url_get_contents(url):
    '''
    Opens a website and read its binary contents (HTTP Response Body)
    '''
    
    # This header is required, otherwise the website will reject the request with a 403 error.
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
 
    # Opens a website and read its
    # binary contents (HTTP Response Body)
 
    # making request to the website
    req = urllib.request.Request(url, None, headers)
    f = urllib.request.urlopen(req)
 
    #reading contents of the website
    return f.read()

def extract_tables_from_url(url: str): 
    # defining the html contents of a URL.
    xhtml = url_get_contents(url).decode('utf-8')

    # Defining the HTMLTableParser object
    p = HTMLTableParser()

    # feeding the html contents in the
    # HTMLTableParser object
    p.feed(xhtml)
    
    print("Extracted {} tables from the url {}".format(len(p.tables), url))

    return p.tables

def extract_tables_from_url_v2(url: str):
    response = url_get_contents(url)
    soup = BeautifulSoup(response,"html.parser")
    
    # Get all the ship list items. 
    ship_lists = soup.find("table", {"class": "shipTableCruise"})
    itin_ids = []

    for ship in ship_lists.find_all("tr"):
        if ship.has_attr("data-row"):
            itin_ids.append(ship["data-row"])
    
    tables = []
    for itin_id in itin_ids:
        response = get_json_itinerary_contents_from_id(itin_id, url)
        xhtml = response.json()["result"]

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)
        tables += p.tables
    return tables
    

def get_ship_name(url: str):
    '''
    Given a URL, returns the string following the very last "/" character. We are assuming
    this is the name of the ship.
    Example: given: "https://www.cruisemapper.com/ships/Norwegian-Breakaway-584", returns "Norwegian-Breakaway-584"
    since it is the string after the last "/".
    '''
    
    return url.split("/")[-1]

def get_csv_file(index: int, url: str):
    return "{}/csv_files/{}/{}-table-{}.csv".format(k_output_directory, get_ship_name(url), get_ship_name(url), index)

def get_sub_csv_file(index: int, url: str):
    return "{}/csv_files/{}/individual_itin/{}-table-{}.csv".format(k_output_directory, get_ship_name(url), get_ship_name(url), index)

def write_to_csv(table: List[List[str]], output_file: str):
    print("\tWriting table to {}".format(output_file))
    output_dir = pathlib.Path(output_file).parents[0]
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for row in table:
            csv_writer.writerow(row)

def process_url(url: str):
    tables = extract_tables_from_url(url)
    index = 0
    for table in tables:
        write_to_csv(table, get_csv_file(index, url))
        index = index + 1

    index = 0
    sub_tables = extract_tables_from_url_v2(url)
    for table in sub_tables:
        write_to_csv(table, get_sub_csv_file(index, url))
        index = index + 1
    return True

In [253]:
def run(num_threads: int = 0, only_norwegian = False):
    all_ship_urls = load_all_ship_urls()
    len(all_ship_urls)
    print(all_ship_urls[0])
    
    if only_norwegian:
        all_ship_urls = list(filter(lambda name: "norwegian" in name.lower(), all_ship_urls))
    
    if num_threads > 1:
        pool = mp.Pool(num_threads)
        results = pool.map(process_url, all_ship_urls)
    else:
        for url in all_ship_urls:
            process_url(url)
    print("Completed!")


In [254]:
run(100, True)
#process_url("https://www.cruisemapper.com/ships/Norwegian-Encore-1518")

Load from cache successful!
Loaded 1503 total ship urls!
https://www.cruisemapper.com/ships/Scenic-Sapphire-1092
Extracted 5 tables from the url https://www.cruisemapper.com/ships/Norwegian-Sky-543
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/Norwegian-Sky-543-table-0.csv
Extracted 4 tables from the url https://www.cruisemapper.com/ships/Norwegian-Dawn-699	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/Norwegian-Sky-543-table-1.csv

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/Norwegian-Dawn-699-table-0.csv
Extracted 4 tables from the url https://www.cruisemapper.com/ships/Norwegian-Spirit-702	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/Norwegian-Sky-543-table-2.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/Norwegian-Dawn-699-table-1.csv
	Writing table to /Users/andrew/Pro

Extracted 9 tables from the url https://www.cruisemapper.com/ships/Norwegian-Bliss-1454

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Epic-642/Norwegian-Epic-642-table-2.csv	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/Norwegian-Joy-1166-table-2.csv	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Prima-2216/Norwegian-Prima-2216-table-3.csv	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/Norwegian-Bliss-1454-table-0.csv



	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/Norwegian-Joy-1166-table-3.csv	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Epic-642/Norwegian-Epic-642-table-3.csv	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Prima-2216/Norwegian-Prima-2216-table-4.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/c

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-20.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-21.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-22.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-23.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-24.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-25.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Dawn-699/individual_itin/Norwegian-Dawn-699-table-26.csv
	Writing table to /Users/andrew/Projects/

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-18.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-19.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-20.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-21.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-22.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-23.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Gem-573/individual_itin/Norwegian-Gem-573-table-24.csv
	Writing table to /Users/andrew/Projects/IWaungTableScr

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-6.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-7.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-8.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-9.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-10.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-11.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Sky-543/individual_itin/Norwegian-Sky-543-table-12.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper

...

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-98.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-99.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-100.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-101.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-102.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-103.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Joy-1166/individual_itin/Norwegian-Joy-1166-table-104.csv
	Writing table to /Users/andrew/Proj

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-40.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-41.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-42.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-43.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-44.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-45.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-46.csv
	Writing tabl

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-98.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-99.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-100.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-101.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-102.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-103.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Bliss-1454/individual_itin/Norwegian-Bliss-1454-table-104.csv
	Writing

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-31.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-32.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-33.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-34.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-35.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-36.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-37.csv

	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-88.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-89.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-90.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-91.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-92.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-93.csv
	Writing table to /Users/andrew/Projects/IWaungTableScraper/csv_files/Norwegian-Encore-1518/individual_itin/Norwegian-Encore-1518-table-94.csv

In [234]:
resp = get_json_itinerary_contents_from_id(1924803, "https://www.cruisemapper.com/ships/Norwegian-Encore-1518")
print(resp.json()["result"])






<table class="table table-bordered cruiseExpand">
    <thead>
        <tr>
            <th>Date / Time</th>
            <th class="labelPort">Port</th>
        </tr>
    </thead>
    <tbody>
                    <tr>
                <td class="date">26 Mar 17:30</td>                <td class="text"><i class="fa fa-flag"></i>  &nbsp;<span class="flag-icon flag-icon-us" title="USA" data-toggle="tooltip" data-placement="left"></span> &nbsp;<strong>Departing</strong> from <a href="https://www.cruisemapper.com/ports/miami-port-40">Miami, Florida</a><a class="itineraryHotelsText" href="https://www.cruisemapper.com/ports/miami-port-40?tab=hotels#hotels"><i class="fa fa-hotel"></i>  hotels</a></td>                </tr>            </tr>
                    <tr>
                <td class="date">28 Mar 07:00 - 16:00</td>                <td class="text"><i class="fa fa-anchor"></i>  &nbsp;<span class="flag-icon flag-icon-do" title="Dominican Rep" data-toggle="tooltip" data-placement="left"></s